In [63]:
import os
import keras
import pandas as pd
import re
import glob
import imageio
import numpy as np

from sklearn.metrics import accuracy_score
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Embedding, LSTM
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop
from IPython.display import Image

## Load data

In [16]:
test_data = pd.read_csv("final_test.csv")
test_data = test_data.iloc[:,1:]

## LSTM Testing

In [17]:
lyric = [x for x in test_data.lyric]
label = [int(x == 'hiphop') for x in test_data.label]

In [18]:
tok = keras.preprocessing.text.Tokenizer(lower=False)

In [19]:
tok.fit_on_texts(lyric)

In [12]:
seq = tok.texts_to_sequences(lyric)

In [29]:
MAXLEN = 2752

In [30]:
pad = keras.preprocessing.sequence.pad_sequences(seq, MAXLEN)

In [35]:
NUM_WORDS = 242623

In [36]:
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=NUM_WORDS, output_dim=8, input_length=MAXLEN, mask_zero=True))
model_lstm.add(LSTM(16, return_sequences=False))
model_lstm.add(Dense(2, activation='softmax'))

In [37]:
model_lstm.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [38]:
model_lstm.load_weights("lstm_model.hd5")

In [44]:
y_pred = model_lstm.predict_classes(pad)
accuracy_score(label, y_pred)

0.3

#### 생각외로 lstm의 성능이 크게 떨어졌다.

#### 이유를 분석해보니 해당 기간에 top100에서 힙합으로 분류된 것은 빡센 힙합이 아닌 발라드 성 힙합을 부른 로꼬, ph-1, 지코, 박재범 등이 포진해있었다.

#### 충분히 그럴 수 있다고 판단된다.

In [77]:
test_data.loc[test_data['label'] == 'hiphop',['singer','label']].head(10)

,singer,label
1,로꼬,hiphop
2,pH-1pH-1,hiphop
7,CoogieCoogie,hiphop
18,기리보이기리보이,hiphop
19,저스디스(JUSTHIS)저스디스(JUSTHIS),hiphop
23,HAON(김하온),hiphop
24,pH-1,hiphop
31,김효은김효은,hiphop
37,나플라(nafla)나플라(nafla),hiphop
45,지코(Zico),hiphop


## CNN Testing

In [46]:
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', input_shape=(200, 200, 3))) # 200 * 200
model.add(BatchNormalization())
model.add(Conv2D(16, (3, 3), activation="relu", kernel_initializer='he_normal')) # 198 * 198
model.add(BatchNormalization())
model.add(Conv2D(16, (3, 3), activation="relu", kernel_initializer='he_normal')) # 196 * 196
model.add(BatchNormalization())
model.add(Conv2D(16, (3, 3), activation="relu", kernel_initializer='he_normal')) # 194 * 194
model.add(BatchNormalization())
model.add(Conv2D(16, (3, 3), activation="relu", kernel_initializer='he_normal')) # 192 * 192
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2))) # 96 * 96

model.add(Conv2D(16, (3,3), activation = "relu", kernel_initializer='he_normal')) # 94 * 94
model.add(BatchNormalization())
model.add(Conv2D(16, (3,3), activation = "relu", kernel_initializer='he_normal')) # 92 * 92
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2))) # 46 * 46

model.add(Conv2D(16, (3,3), activation = "relu", kernel_initializer='he_normal')) # 44 * 44
model.add(BatchNormalization())
model.add(Conv2D(16, (3,3), activation = "relu", kernel_initializer='he_normal')) # 42 * 42
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2))) # 21 * 21

model.add(Flatten())
model.add(Dense(50, activation = "relu", kernel_initializer='he_normal'))
model.add(Dense(1, activation = "sigmoid"))

In [47]:
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam())

In [48]:
model.load_weights("cnn_acc77_batchnorm.hd5")

In [50]:
import os
os.getcwd()

'/home/ubunt/3-2/Unstructured/project/script'

In [51]:
folder_path = '/home/ubunt/3-2/Unstructured/project/data/final_test/image/*'

In [64]:
#####1000폴더 6000장 기준 8초 소요 
dataset=[] ###모든 이미지가 저장됨(폴더구분없이)

for image_path in sorted(glob.glob(folder_path + "*")): #####각 라벨 폴더에서 이미지 가져오가 
    im = imageio.imread(image_path)
    dataset.append(im)

dataset = np.array(dataset)

In [69]:
pred = model.predict_classes(dataset)

#### 의외로 가사보다는 이미지에서 힙합을 더 잘 맞추었다.

In [70]:
accuracy_score(label, pred)

0.79

# 결론.

### 우선 가지고있는 데이터 8000개로 도출할 수 있는 결론은 이미지는 분류에 효과가 있었고 텍스트는 그렇지 않았습니다.

### 군집화는 2개의 군집을 할당해도 장르별로 묶이는 것 보다는 영어와 한글로 묶이는 경향이 보였습니다.

### 물론 최종 예측 데이터 100개가 힙합이 빡센 힙합이 아니라는점에서 유의미한 결론은 아니라고 할 수 있습니다.

### 추가사항으로 힙합에도 정말 빡센 힙합, 발라드성 힙합이 존재하는데 만약 train data를 십만개 단위로 늘려서

### 보다 더 많은 과정을 담을 수 있다면 더 좋은 성능이 나올 것으로 기대됩니다.